In [65]:
import enum

class Data:
    data: list[int] = []

    self: list[int] = []
    other: list[int] = []
    grad: list[int] = []
    op: list[int] = [] # fn pointer to operation
    fn: list[int] = [] # fn pointer for backward fn

    def __repr__(self) -> str:
        print(self.self, self.other, self.grad, self.op, self.fn, sep="\n")
        return ""

storage = Data()


class Operations(enum.Enum):
    add = lambda x,y : x + y
    mul = lambda x,y : x * y
    pow = lambda x,y : x ** y


class Scalar:

    def __init__(self, data = 0, grad = 0, parents = []):
        self.data = float(data)
        self.grad = float(grad)
        self.parents = parents
        self.backward_fn = lambda x: None

    def __add__(self, other):
        if not isinstance(other, Scalar):
            other = Scalar(other)
        
        def backward(d_out):
            self.grad += d_out
            other.grad += d_out
            return d_out, d_out

        out = Scalar(self.data + other.data, parents = [self, other])
        out.backward_fn = backward

        storage.data.append(Operations.add(self.data, other.data))

        storage.self.append(self.data)
        storage.other.append(other.data)
        storage.grad.append(self.grad)
        storage.op.append(Operations.add)
        storage.fn.append(backward)
      
        return out

    def __mul__(self, other):
        if not isinstance(other, Scalar):
            other = Scalar(other)

        def backward(d_out):
            self.grad += other.data * d_out
            other.grad += self.data * d_out
            return other * d_out, self * d_out

        out = Scalar(self.data * other.data, parents = [self, other])
        out.backward_fn = backward
        
        storage.data.append(Operations.mul(self.data, other.data))
        
        storage.self.append(self.data)
        storage.other.append(other.data)
        storage.grad.append(self.grad)
        storage.op.append(Operations.mul)
        storage.fn.append(backward)

        return out

    def __pow__(self, other):
        
        def backward(d_out):
            dx_dy = other * (self.data ** (other - 1)) * d_out
            self.grad += dx_dy
            return dx_dy

        out = Scalar(self.data ** other, parents = [self])
        out.backward_fn = backward
        
        storage.data.append(Operations.pow(self.data, other))

        storage.self.append(self.data)
        storage.other.append(other)
        storage.grad.append(self.grad)
        storage.op.append(Operations.pow)
        storage.fn.append(backward)
        
        return out

    # 
    
    def __neg__(self):  # -self
        return self * -1

    def __radd__(self, other):  # other + self
        return self + other

    def __rmul__(self, other):  # other * self
        return self * other

    def __sub__(self, other):  # self - other
        return self + (-other)

    def __rsub__(self, other):  # other - self
        return other + (-self)

    def __truediv__(self, other):  # self / other
        return self * (other**-1)

    def __rtruediv__(self, other):  # other / self
        return other * (self**-1)

    def __repr__(self):
        return "Scalar(data=%s,grad=%s)" % (self.data, self.grad)

    def topological_sort(self):
        vis = set()
        order = list()
        stk = [self]
        while stk:
            node = stk.pop()
            if node in vis:
                continue
            vis.add(node)
            stk.extend(node.parents)
            order.append(node) 
        return order

    def backward(self):
        order = self.topological_sort()

        self.grad = 1.0
        for node in order:
            node.backward_fn(node.grad)
        return

    def build(self):
        """ Transfer OOP data from each Scalar instance to DOD array"""

        res = 0
        for i in range(len(storage.self)):
            res += storage.op[i](storage.self[i], storage.other[i])

        print("DOD", res)
        return

a, b, c, d, e = [ Scalar(i) for i in range(2, 7) ]
print("Created Scalars: ", a,b,c,d,e)

result = a * b + c - d / e
result.backward()

print("Result Scalar: ", result)
print("Updated Scalars: ", a,b,c,d,e, sep="\n")

result.build()
storage.self.__len__(), storage.fn.__len__(), storage.op.__len__()
storage

Created Scalars:  Scalar(data=2.0,grad=0.0) Scalar(data=3.0,grad=0.0) Scalar(data=4.0,grad=0.0) Scalar(data=5.0,grad=0.0) Scalar(data=6.0,grad=0.0)
Result Scalar:  Scalar(data=9.166666666666666,grad=1.0)
Updated Scalars: 
Scalar(data=2.0,grad=3.0)
Scalar(data=3.0,grad=2.0)
Scalar(data=4.0,grad=1.0)
Scalar(data=5.0,grad=-0.16666666666666666)
Scalar(data=6.0,grad=0.1388888888888889)
DOD 25.0
[2.0, 6.0, 6.0, 5.0, 0.8333333333333333, 10.0, -1.0, 0.8333333333333333, 0.16666666666666666, 5.0, 3.0, 2.0]
[3.0, 4.0, -1, 0.16666666666666666, -1.0, -0.8333333333333333, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8333333333333333, -1.0, -5.0, -0.16666666666666666, 2.0, 3.0]
[<function Operations.<lambda> at 0x7fdbe6ae1a80>, <function Operations.<lambda> at 0x7fdbe6ae1c60>, <function Operations.<lambda> at 0x7fdbe6ae1d00>, <function Operations.<lambda> at 0x7fdbe6ae1a80>, <function Operations.<lambda> at 0x7fdbe6ae1a80>, <function Operations.<lambda> at 0x7fdbe6ae1c60>, <functi